# SuperPoderosas

Índice:

1. Gestión de datos vacíos, imputación y otliers
2. Numeración de las variables categóricas
3. Separación de datos de train en train_final y train_val
4. Normalizar variables numéricas
5. Estudio y extracción de características
6. Entrenamiento de modelos
7. Validación con datos de validación estudio de modelos
8. Uso de los datos de test y pasarlo a kaggle

    8. Reversión de modificaciones
9. Conclusiones del estudio

## 1. Gestión de datos vacíos, imputación y otliers

Carga de datos:

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics, preprocessing
#warnings.filterwarnings(action='ignore')                  # Turn off the warnings.
%matplotlib inline

In [7]:
train_dataset = pd.read_csv('train.csv', header='infer')
test_dataset = pd.read_csv('test.csv', header='infer')

train_dataset.columns
#test_dataset



Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

Id.
MSSubClass.
MSZoning.
LotFrontage.
LotArea.
Street.
Alley.
LotShape.
LandContour.
Utilities.
LotConfig.
LandSlope.
Neighborhood.
Condition1.
Condition2.
BldgType.
HouseStyle.
OverallQual.
OverallCond.
YearBuilt.
YearRemodAdd.
RoofStyle.
RoofMatl.
Exterior1st.
Exterior2nd.
MasVnrType.
MasVnrArea.
ExterQual.
ExterCond.
Foundation.
BsmtQual.
BsmtCond.
BsmtExposure.
BsmtFinType1.
BsmtFinSF1.
BsmtFinType2.
BsmtFinSF2.
BsmtUnfSF.
TotalBsmtSF.
Heating.
HeatingQC.
CentralAir.
Electrical.
1stFlrSF.
2ndFlrSF.
LowQualFinSF.
GrLivArea.
BsmtFullBath.
BsmtHalfBath.
FullBath.
HalfBath.
BedroomAbvGr.
KitchenAbvGr.
KitchenQual.
TotRmsAbvGrd.
Functional.
Fireplaces.
FireplaceQu.
GarageType.
GarageYrBlt.
GarageFinish.
GarageCars.
GarageArea.
GarageQual.
GarageCond.
PavedDrive.
WoodDeckSF.
OpenPorchSF.
EnclosedPorch.
3SsnPorch.
ScreenPorch.
PoolArea.
PoolQC.
Fence.
MiscFeature.
MiscVal.
MoSold.
YrSold.
SaleType.
SaleCondition.
SalePrice.

In [36]:
#  Busca valores nulos en training.
train_data_null_values = train_dataset.isnull().sum(axis=0)

train_null_val = train_data_null_values[train_data_null_values > 0 ]
#print(train_null_val)

#print("-------------")
#  Busca valores nulos en training.
test_data_null_values = test_dataset.isnull().sum(axis=0)

test_null_val = test_data_null_values[test_data_null_values > 0 ]
#print(test_null_val)

total_null_dataset = train_null_val.fillna(0).add(test_null_val.fillna(0), fill_value=0)
total_null_dataset

# 

Alley           2721.0
BsmtCond          82.0
BsmtExposure      82.0
BsmtFinSF1         1.0
BsmtFinSF2         1.0
BsmtFinType1      79.0
BsmtFinType2      80.0
BsmtFullBath       2.0
BsmtHalfBath       2.0
BsmtQual          81.0
BsmtUnfSF          1.0
Electrical         1.0
Exterior1st        1.0
Exterior2nd        1.0
Fence           2348.0
FireplaceQu     1420.0
Functional         2.0
GarageArea         1.0
GarageCars         1.0
GarageCond       159.0
GarageFinish     159.0
GarageQual       159.0
GarageType       157.0
GarageYrBlt      159.0
KitchenQual        1.0
LotFrontage      486.0
MSZoning           4.0
MasVnrArea        23.0
MasVnrType      1766.0
MiscFeature     2814.0
PoolQC          2909.0
SaleType           1.0
TotalBsmtSF        1.0
Utilities          2.0
dtype: float64

Se procede a realizar un estudio de las siguientes variables para determinar qué realizar con los valores faltantes. Para ello se procederá a eliminar todas las filas con una tasa de NaN inferior al 1% (es decir, menos de 29 valores nulos)

https://www.kaggle.com/code/asyaaatru/sale-price-as-detailed-as-possible

Las columnas con 29 o menos NaN son las siguientes. Se procederá a la eliminación completa de la fila correspondiente al valor faltante: 
- BsmtFinSF1         1.0 - Filas eliminadas (Cantidad menor al umbral)
- BsmtFinSF2         1.0 - Filas eliminadas (Cantidad menor al umbral)
- BsmtFullBath       2.0 - Filas eliminadas (Cantidad menor al umbral)
- BsmtHalfBath       2.0 - Filas eliminadas (Cantidad menor al umbral)
- BsmtUnfSF          1.0 - Filas eliminadas (Cantidad menor al umbral)
- Electrical         1.0 - Filas eliminadas (Cantidad menor al umbral)
- Exterior1st        1.0 - Filas eliminadas (Cantidad menor al umbral)
- Exterior2nd        1.0 - Filas eliminadas (Cantidad menor al umbral)
- Functional         2.0 - Filas eliminadas (Cantidad menor al umbral)
- GarageArea         1.0 - Filas eliminadas (Cantidad menor al umbral)
- GarageCars         1.0 - Filas eliminadas (Cantidad menor al umbral)
- KitchenQual        1.0 - Filas eliminadas (Cantidad menor al umbral)
- MSZoning           4.0 - Filas eliminadas (Cantidad menor al umbral)
- MasVnrArea        23.0 - Filas eliminadas (Cantidad menor al umbral)
- SaleType           1.0 - Filas eliminadas (Cantidad menor al umbral)
- TotalBsmtSF        1.0 - Filas eliminadas (Cantidad menor al umbral)
- Utilities          2.0 - Filas eliminadas (Cantidad menor al umbral)

Una vez filtrados, estudiaremos los datos con NaN restantes uno a uno, decidiendo si eliminar o imputar:

- Alley           2721.0 - Considerado como "Cantidad = 0"
- BsmtCond          82.0 - Considerado como "Cantidad = 0"
- BsmtExposure      82.0 - Considerado como "Cantidad = 0"
- BsmtFinType1      79.0 - Considerado como "Cantidad = 0"
- BsmtFinType2      80.0 - Considerado como "Cantidad = 0"
- BsmtQual          81.0 - Considerado como "Cantidad = 0"
- Fence           2348.0 - Considerado como "Cantidad = 0"
- FireplaceQu     1420.0 - Considerado como "Cantidad = 0"
- GarageCond       159.0 - Considerado como "Cantidad = 0"
- GarageFinish     159.0 - Considerado como "Cantidad = 0"
- GarageQual       159.0 - Considerado como "Cantidad = 0"
- GarageType       157.0 - Considerado como "Cantidad = 0"
- GarageYrBlt      159.0 - Considerado como "Cantidad = 0"
- LotFrontage      486.0 - Valor faltante
- MasVnrType      1766.0 - Valor faltante
- MiscFeature     2814.0 - Considerado como "Cantidad = 0"
- PoolQC          2909.0 - Considerado como "Cantidad = 0"

De las variables con NaN, solo en "LotFrontage" y "MasVnrType" se han considerado que los NaN hacen referencia a datos vacíos. Se procederán a descartar las columnas por las siguientes razones:
 - En "MasVnrType" hay más de la mitad de datos faltantes, por lo que descarta.
 - En "LotFrontage" hay un 16.7% de datos faltantes, por lo que se procedió a analizar su relevancia. Tras un riguroso análisis se consideró poco relevante y se eliminó del modelo final(Fuente: la de los deseos)

In [24]:
train_dataset.sort_values(by='LotFrontage')["LotFrontage"]

232     21.0
837     21.0
235     21.0
1029    21.0
1038    21.0
        ... 
1429     NaN
1431     NaN
1441     NaN
1443     NaN
1446     NaN
Name: LotFrontage, Length: 1460, dtype: float64

In [ ]:
train_dataset["LotFrontage"]